# OpenAI API 사전 준비

In [1]:
# 패키지 설치
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.14.0
    Uninstalling openai-2.14.0:
      Successfully uninstalled openai-2.14.0


In [2]:
# 환경 변수 준비 (왼쪽 끝 키 아이콘으로 OPENAI_API_KEY 설정)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

# 클라이언트 준비
client = OpenAI()

# Function Calling 실행

In [4]:
import requests

# 외부 함수 정의
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [5]:
# 툴 리스트 준비
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "지정된 좌표의 현재 온도를 섭씨로 가져옵니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

In [7]:
# 메시지 리스트 준비
messages = [
    {"role": "user", "content": "서울의 기온은?"}
]

In [8]:
# 추론 실행
response = client.responses.create(
    model="gpt-5",
    input=messages,
    tools=tools  # 툴
)
print(response.output_text)

In [9]:
tool_call = None

# 출력 확인
for item in response.output:
    # function_call
    if item.type == "function_call":
        print("==function_call==")
        print(item)
        tool_call = item

==function_call==
ResponseFunctionToolCall(arguments='{"latitude": 37.5665, "longitude": 126.978}', call_id='call_BxOv95nJGuuRBL5W77NrdjEi', name='get_weather', type='function_call', id='fc_0f54f4cc7fae71d20069666330e4a0819286f9e4a5f0a2e942', status='completed')


In [10]:
import json

# 외부 함수 실행
args = json.loads(tool_call.arguments)
result = get_weather(args["latitude"], args["longitude"])
print(result)

-8.4


In [11]:
# 메시지 리스트 준비 (외부 함수 호출 결과)
messages2 = [
    {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
]

# 추론 실행
response2 = client.responses.create(
    model="gpt-5",
    input=messages2,
    tools=tools,
    previous_response_id=response.id, # 대화 상태 관리
)
print(response2.output_text)

서울(시청 인근) 현재 기온은 약 -8.4°C입니다.
